# <div align='center'> 分类任务测试 </div>

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from k12libs.utils.nb_easy import k12ai_run_project

In [3]:
network = 'base_model'
dataset = 'mnist'

In [4]:
project = k12ai_run_project(debug=True, tb_port=9002,
                            framework='k12cv', task='cls', 
                            network=network, dataset=dataset)

Box(children=(Box(children=(VBox(children=(Tab(children=(VBox(children=(HBox(children=(Text(value='cls', conti…